# Word2GM Training Data Pipeline

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare a Google 5gram corpora for Word2GM skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` NVMe storage
2. **Processing**: TensorFlow-native filtering, vocabulary building, and triplet generation
3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [6]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_data_preprocessing_notebook, enable_autoreload

# Enable autoreload for development
enable_autoreload()

# Set up environment (CPU-only for data preprocessing)
env = setup_data_preprocessing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
batch_prepare_training_data = env['batch_prepare_training_data']
print_resource_summary = env['print_resource_summary']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: CPU-only</pre>

<pre>Data preprocessing environment ready!</pre>

## Print Resource Summary

In [7]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: gr043.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Requested partitions: rtx8000,v100,a100_2,a100_1,h100_1
   Running on: SSH failed: Host key verification failed.
   Job ID: 63296830
   Node list: gr043

GPU Information:
   CUDA GPUs detected: 1
   GPU 0: Quadro RTX 8000
      Memory: 0.5/45.0 GB (44.5 GB free)
      Temperature: 36°C
      Utilization: GPU 0%, Memory 0%

TensorFlow GPU Detection:
   TensorFlow detects 0 GPU(s)
   Built with CUDA: True
============================================================</pre>

## Prepare Corpora

Here, we run the data-preparation pipeline from start to finish — reading preprocessed ngram corpora, generating all valid triplets, extracting the vocabulary, and saving the triplets and vocabulary as `tfrecord` files.

### Options for Data Preparation

You can control which years are processed and how the batch preparation runs by adjusting the arguments to `batch_prepare_training_data`:

**Ways to specify years:**
- `year_range="2010"` — Process a single year (e.g., only 2010).
- `year_range="2010,2012,2015"` — Process a comma-separated list of years.
- `year_range="2010-2015"` — Process a range of years, inclusive (2010 through 2015).
- `year_range="2010,2012-2014,2016"` — Combine individual years and ranges (2010, 2012, 2013, 2014, 2016).

**Other options:**
- `compress` — If `True`, output TFRecords are gzip-compressed. If `False`, output is uncompressed.
- `show_progress` — If `True`, display a progress bar for each year.
- `show_summary` — If `True`, print a summary of the processed data for each year.
- `use_multiprocessing` — If `True`, process years in parallel using multiple CPU cores (recommended for large datasets).

See the function docstring or source for more advanced options.

In [11]:
# Configuration
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

# Process years with multiprocessing (CPU-only mode configured in cell 2)
results = batch_prepare_training_data(
    corpus_dir=corpus_dir,
    year_range="1701-1800",
    compress=False,
    show_progress=True,
    show_summary=True,
    use_multiprocessing=True
)


PARALLEL BATCH PROCESSING
Processing 100 years
Using 14 parallel workers
Estimated speedup: 14.0x


1702 complete (1/100): 63 triplets, 75 vocab, 0.9s
1712 complete (2/100): 23 triplets, 36 vocab, 1.0s
1713 complete (3/100): 257 triplets, 210 vocab, 1.1s
1707 complete (4/100): 117 triplets, 108 vocab, 1.1s
1701 complete (5/100): 881 triplets, 542 vocab, 1.5s
1706 complete (6/100): 737 triplets, 483 vocab, 1.6s
1717 complete (7/100): 314 triplets, 312 vocab, 0.6s
1709 complete (8/100): 738 triplets, 465 vocab, 1.7s
1708 complete (9/100): 905 triplets, 564 vocab, 1.8s
1703 complete (10/100): 1,589 triplets, 794 vocab, 2.2s
1718 complete (11/100): 361 triplets, 321 vocab, 1.1s
1714 complete (12/100): 1,601 triplets, 834 vocab, 2.3s
1715 complete (13/100): 1,497 triplets, 815 vocab, 1.4s
1711 complete (14/100): 1,991 triplets, 983 vocab, 2.6s
1721 complete (15/100): 1,071 triplets, 644 vocab, 1.2s
1705 complete (16/100): 4,010 triplets, 1,568 vocab, 3.5s
1716 complete (17/100): 2,749 triplets, 1,110 vocab, 2.5s
1704 complete (18/100): 4,645 triplets, 1,738 vocab, 3.6s
1725 complete (19/1